##custom_kernel_function (SVM)
- custom kernel
- built-in kernel
- compare custom vs built-in
- hyperparameter tuning
 

In [ ]:

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
import sklearn
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.model_selection import GridSearchCV

Read Dataset File

In [ ]:
trainX = pd.read_csv("trainX.csv",header=None)
trainY = pd.read_csv("trainY.csv",header=None)
testX = pd.read_csv("testX.csv",header=None)
testY = pd.read_csv("testY.csv",header=None)

Build Custom Kernel Function 
- sigmoid
- poly
- rbf 

In [ ]:
## Sigmoid : Tanh

def my_sigmoid(x1,x2):
    return np.tanh(np.dot(x1, x2.T))

clf = SVC(random_state=2011, kernel=my_sigmoid)
clf = clf.fit(trainX, trainY.values.ravel()) # not to get warning message 
yp = clf.predict(testX)
print('** my_sigmoid')
print("Accuracy:", accuracy_score(testY, yp))
print("Macro F1:", f1_score(testY, yp, average='macro'))
sigmoid_acc= accuracy_score(testY, yp)
sigmoid_f1 = f1_score(testY, yp, average='macro')

** my_sigmoid
Accuracy: 0.5045
Macro F1: 0.5044225660259416


In [ ]:
## Poly : (x1*x2')**2

def my_poly(x1,x2):
    return np.power(np.dot(x1, x2.T), 2)

clf = SVC(random_state=2011, kernel=my_poly)
clf = clf.fit(trainX, trainY.values.ravel()) 
yp = clf.predict(testX)
print('** my_poly')
print("Accuracy:", accuracy_score(testY, yp))
print("Macro F1:", f1_score(testY, yp, average='macro'))
poly_acc= accuracy_score(testY, yp)
poly_f1 = f1_score(testY, yp, average='macro')

** my_poly
Accuracy: 0.692
Macro F1: 0.6918641120734244


In [ ]:
## RBF

def my_rbf(x1, x2):

    a,b,n= x1.shape[0], x2.shape[0], x1.shape[1]
    arr = np.zeros(shape=(a,b))                # empty dataframe
    A,B = np.diag(np.dot(x1 ,x1.T)), np.diag(np.dot(x2,x2.T))
    AB = np.dot(x1, x2.T)

    for i in tqdm(range(0,a)):   #check progress bar
        for j in range(0,b):
            arr[i,j] = (A[i]+B[j]) - 2*AB[i][j]

    # arr = np.tile(np.diag(x1 @x1.T), (a, b)) +np.tile (np.diag(x2@x2.T).T,(a,b) -2*np.dot(x1 @ x2.T) 
    # : x^2+x^2 -2*xy  =>  can calculate the result wuthout for loop , but limited RAM

    gamma = 1/a                           # gamma = 'auto'  in buit-in module
    return np.exp(-gamma *arr)

clf = SVC(random_state=2011, kernel = my_rbf)
clf.fit(trainX, trainY.values.ravel())
yp = clf.predict(testX)

print("** my_rbf")
print("Accuracy:", accuracy_score(testY, yp))
print("Macro F1:", f1_score(testY, yp, average='macro'))
rbf_acc= accuracy_score(testY, yp)
rbf_f1 = f1_score(testY, yp, average='macro')

  0%|          | 0/11000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

** my_rbf
Accuracy: 0.84275
Macro F1: 0.8426755757145366


In [ ]:
custom_df = pd.DataFrame([[sigmoid_acc,poly_acc,rbf_acc], [sigmoid_f1,poly_f1,rbf_f1]] ,index=['[acc]','[ f1]'], columns = ['my_sigmoid','my_poly','my_rbf'])
print(custom_df)

       my_sigmoid   my_poly    my_rbf
[acc]    0.504500  0.692000  0.842750
[ f1]    0.504423  0.691864  0.842676


Built-in Results
- sigmoid
- poly
- rbf

In [ ]:
param_grid = {'kernel': ['sigmoid','poly','rbf']}  #list of built-in kernels
 
grid = GridSearchCV(SVC(), param_grid, scoring='f1_macro',n_jobs=-1)  #check f1_marco score 

grid.fit(trainX, trainY.values.ravel())
builtin_df = pd.DataFrame(grid.cv_results_)       #get result dataframe 
print(builtin_df [["param_kernel","mean_test_score", "mean_fit_time"]])

  param_kernel  mean_test_score  mean_fit_time
0      sigmoid         0.507471      12.572975
1         poly         0.509278      11.608768
2          rbf         0.850630      11.508695


Hyperparameter Tuning

In [ ]:
acc =[]
f1 = []

In [ ]:
## gamma

def my_rbf(x1, x2):

    a,b,n= x1.shape[0], x2.shape[0], x1.shape[1]
    arr = np.zeros(shape=(a,b))                # empty dataframe
    A,B = np.diag(np.dot(x1 ,x1.T)), np.diag(np.dot(x2,x2.T))
    AB = np.dot(x1, x2.T)

    for i in tqdm(range(0,a)):   #check progress bar
        for j in range(0,b):
            arr[i,j] = (A[i]+B[j]) - 2*AB[i][j]


    #gamma = 1/a                           # clf1 gamma =  1/n_features       : 'auto'  in buit-in module
    #gamma = 1/(a*np.array(x1).var())      # clf2 gamma =  1/(n_features*var) : 'scale' in buit-in module 
    #gamma = 0.1                           # clf3 gamma =  0.1
    gamma = 1                             # clf4 gamma =  1 

    return np.exp(-gamma *arr)

clf = SVC(random_state=2011, kernel = my_rbf)
clf.fit(trainX, trainY.values.ravel())
yp = clf.predict(testX)
acc.append(accuracy_score(testY, yp))
f1.append(f1_score(testY, yp, average='macro'))


  0%|          | 0/11000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

In [ ]:
gamma_df = pd.DataFrame([acc,f1] ,index=['[acc]','[ f1]'], columns = ['1/n','1/(var*n)','0.1','1'])

print(gamma_df)

            1/n  1/(var*n)       0.1         1
[acc]  0.842750   0.491000  0.487250  0.487250
[ f1]  0.842676   0.397465  0.327618  0.327618


In [ ]:
## C
acc=[]
f1=[]

for c in [0.1,0.5,1,5,50,100]:
    clf = SVC(random_state=2011, kernel = my_rbf,C=c)
    clf.fit(trainX, trainY.values.ravel())
    yp = clf.predict(testX)
    acc.append(accuracy_score(testY, yp))
    f1.append(f1_score(testY, yp, average='macro'))

  0%|          | 0/11000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/11000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/11000 [00:00<?, ?it/s]

...

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/11000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/11000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

In [ ]:
C_df = pd.DataFrame([acc,f1], index=['[acc]','[ f1]'], columns = ['0.1','0.5','1','5','50','100'])
print(C_df)

            0.1       0.5         1         5        50       100
[acc]  0.787250  0.842250  0.842750  0.822250  0.624000  0.592000
[ f1]  0.787154  0.842175  0.842676  0.822041  0.609058  0.567063


In [ ]:
## result

print('** gamma')
print(gamma_df)
print()
print('** C value')
print(C_df)

** gamma
            1/n  1/(var*n)       0.1         1
[acc]  0.842750   0.491000  0.487250  0.487250
[ f1]  0.842676   0.397465  0.327618  0.327618

** C value
            0.1       0.5         1         5        50       100
[acc]  0.787250  0.842250  0.842750  0.822250  0.624000  0.592000
[ f1]  0.787154  0.842175  0.842676  0.822041  0.609058  0.567063
